In [ ]:
import pandas as pd
import numpy as np
import scanpy as sc
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
import pathlib as pl
import os

In [ ]:
from scipy.sparse import csr_matrix

In [ ]:
from statannotations.Annotator import Annotator

In [ ]:
from tqdm.notebook import tqdm

In [ ]:
def pretty_ax(ax):
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)
    ax.tick_params(
        axis='both',  
        which='both',      
        bottom=True,     
        top=False,
        left=False,
        labelbottom=True,
        labelleft = True)
    ax.spines["bottom"].set_linewidth(1.5)
    ax.spines["left"].set_linewidth(1.5)

In [ ]:
signature_dir = pl.Path("/add/path/here/fibroblast/")

fibroblast_markers = {}
for s in (signature_dir).iterdir():
    sig = s.stem
    fibroblast_markers[sig] = pd.read_csv(s,index_col=0).set_index("names")
    fibroblast_markers[sig] = fibroblast_markers[sig][~fibroblast_markers[sig].index.str.startswith(("MT-","RPS","RPL"))]

# CAF atlas

In [ ]:
metadata = pd.read_csv("/add/path/here/GSE210347_meta.txt.gz", sep="\t",index_col=0)
adata = sc.read_h5ad("/add/path/here/GSE210347_counts.h5ad")
adata.obs = pd.concat([adata.obs,metadata],axis=1)

In [ ]:
fibadata = adata[adata.obs.celltype=="Fibroblast"].copy()
fibadata.__dict__['_raw'].__dict__['_var'] = fibadata.__dict__['_raw'].__dict__['_var'].rename(columns={'_index': 'features'})

In [ ]:
fibadata.layers["counts"] = fibadata.X.copy()
sc.pp.normalize_total(fibadata)
sc.pp.log1p(fibadata)

sc.tl.pca(fibadata)

sc.external.pp.harmony_integrate(fibadata, key="SampleID", max_iter_harmony=20)

In [ ]:
sc.pp.neighbors(fibadata, use_rep="X_pca_harmony")

sc.tl.umap(fibadata)

In [ ]:
mTFs = ["BNC2","RUNX1","RUNX2","PRRX1",
       ]

In [ ]:
for sig, genes in fibroblast_markers.items():
    print(sig)
    sc.tl.score_genes(adata=fibadata,
                        gene_list=list(np.setdiff1d(genes.head(100).index,mTFs)), 
                        score_name=f"{sig}_score",use_raw=False)

In [ ]:
sc.pl.umap(fibadata, color=['Inflammatory CAF_score', 'Fibroblast_score'] ,ncols=2, frameon=False)

In [ ]:
sc.pl.umap(fibadata, color=['group',"tissue"] ,ncols=2, frameon=False)

In [ ]:
fibadata.write_h5ad("/add/path/here/GSE210347_fibroblast_counts.h5ad")

In [ ]:
os.makedirs("figures/external", exist_ok=True)

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(1.5,1.5))

pairs = [("Adjacent","Normal"),
         ("Adjacent","Tumor"),
         ("Normal","Tumor")]

sns.boxplot(data=fibadata.obs,x="group",
            y="Inflammatory CAF_score", 
            order=["Normal","Adjacent","Tumor"],
            ax=ax)
pretty_ax(ax)
ax.set_xlabel("")
ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha="right")
ax.set_ylabel("Inflammatory CAF\nscore")

annot = Annotator(
    ax,
    pairs=pairs,
    data=fibadata.obs,x="group",
            y="Inflammatory CAF_score", 
            order=["Normal","Adjacent","Tumor"]
)
annot.configure(
    test="Mann-Whitney",
    loc="inside",
    text_format="star",
    show_test_name=False,
    verbose=2,
    comparisons_correction=None,
    fontsize=10,
)
annot.apply_test()
_, test_results = annot.annotate()

fig.savefig("figures/external/inflamm_CAF_score_type.svg", dpi=200, bbox_inches="tight")

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(1.5,1.5))

pairs = [("Adjacent","Normal"),
         ("Adjacent","Tumor"),
         ("Normal","Tumor")]

sns.boxplot(data=fibadata.obs,x="group",
            y="Fibroblast_score", 
            order=["Normal","Adjacent","Tumor"],
            ax=ax)
pretty_ax(ax)
ax.set_xlabel("")
ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha="right")
ax.set_ylabel("Fibroblast\nscore")

annot = Annotator(
    ax,
    pairs=pairs,
    data=fibadata.obs,x="group",
            y="Fibroblast_score", 
            order=["Normal","Adjacent","Tumor"]
)
annot.configure(
    test="Mann-Whitney",
    loc="inside",
    text_format="star",
    show_test_name=False,
    verbose=2,
    comparisons_correction=None,
    fontsize=10,
)
annot.apply_test()
_, test_results = annot.annotate()

fig.savefig("figures/external/fibroblast_score_type.svg", dpi=200, bbox_inches="tight")